# plot_lwp_rad_flux.ipynb

Code to plot time series of liquid water path and LW and SW downwelling surface flux. Also plots distributions.

In [ ]:
import xarray as xr
import pandas as pn
import numpy as np
import numpy.ma as ma
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import capricorn_figure_props as fprops
import capricorn_functions as cfn
from capricorn_functions import get_hist_bins
import capricorn_constants as const
from importlib import reload
import simulations
reload(simulations)
from simulations import simulations
from glob import glob
reload(cfn)
reload(const)
reload(fprops)


## Load CAP measurements from file

In [ ]:
# LWP from microwave radiometer
lwp_files = glob('data/CAPRICORN/*_oe_Capricorn2*')
jday_const = 1721425.5  # number of days from 1 Jan 4712 BC to 1 Jan 1 AD
jday_ref_time = dt.datetime(1,1,1)
lwp_da = []
rain_rate_da = []
for file in lwp_files:
    ds = xr.open_dataset(file)
    time_name = 'mpl_mjday'
    lwp_name = 'mwr_liquid_water_path'
    time = np.array([jday_ref_time + dt.timedelta(days=(X-jday_const))
                      for X in ds[time_name].values])
    lwp_da.append(xr.DataArray(
        data = ds[lwp_name],
        coords = {'time': ('events', time)}
    ))
    rain_rate_da.append(xr.DataArray(
        data = ds['rain_rate'],
        coords = {'time': ('events', time)}
    ))
cap_lwp = xr.concat(lwp_da, dim='events')
cap_rain_rate = xr.concat(rain_rate_da, dim='events')

# LW and SW downwelling fluxes from CAP underway data
# https://www.marine.csiro.au/data/trawler/survey_details.cfm?survey=IN2018_V01
# last accessed 26/11/2024
filename='data/CAPRICORN/in2018_v01uwy1min.csv'
nav_df = pn.read_csv(filename, parse_dates=True, low_memory=False)
# save times as dt objects (easier to manipulate)
cap_nav_times = []
fmt_date = '%d-%b-%Y %H:%M:%S'
for i in np.arange(nav_df["date"].shape[0]):
    time_str = f'{nav_df["date"][i]} {nav_df["time"][i]}'
    cap_nav_times.append(dt.datetime.strptime(time_str, fmt_date))
cap_nav_times = np.array(cap_nav_times)
radiometer_stbd = pn.to_numeric(nav_df["stbdRadiometer(W/m^2)"], errors='coerce')
radiometer_port = pn.to_numeric(nav_df["portRadiometer(W/m^2)"], errors='coerce')
pyranometer_stbd = pn.to_numeric(nav_df["stbdPyranometer(W/m^2)"], errors='coerce')
pyranometer_port = pn.to_numeric(nav_df["portPyranometer(W/m^2)"], errors='coerce')
radiometer = ma.mean([radiometer_stbd, radiometer_port], axis=0)
pyranometer = ma.mean([pyranometer_stbd, pyranometer_port], axis=0)
cap_surface_rad_flux = xr.Dataset(
    data_vars={'lw_down': ('time', radiometer),
               'sw_down': ('time', pyranometer)},
    coords={'time': cap_nav_times}
)

# take 1 hr means
cap_lwp_1hr = cap_lwp.resample(time="1h").reduce(ma.mean)
cap_rain_rate_1hr = cap_rain_rate.resample(time="1h").reduce(ma.mean)
cap_rad_1hr = cap_surface_rad_flux.resample(time="1h").reduce(ma.mean)


## Load model output

In [ ]:
model_lwp = []
model_iwp = []
model_lwc = []
model_sw = []
model_lw = []
model_rain_rate = []
i_fig_sim = [0,1]
for s,sim in enumerate(simulations):
    print(sim.code)
    # load air density in kg m-3, to convert mass mixing ratio to concentrations
    rho = cfn.load_air_density(sim.code, grid=True)
    
    liquid_mass = xr.open_dataarray(f'data/model/{sim.code}_m01s00i254_grid_CAP.nc')
    lwc = liquid_mass*rho.air_density
    model_lwc.append(lwc)

    # load liquid cloud fraction
    cfliq = xr.open_dataarray(f'data/model/{sim.code}_m01s00i267_grid_CAP.nc')
    cfliq_nonzero = ma.masked_less_equal(cfliq.values, 0)

    # scale LWC by cloud fraction to get in-cloud values
    lwc_cloud = lwc/cfliq_nonzero

    # calculate model z coord and z widths
    altitude = lwc_cloud.TH_1_60_eta_theta.values*const.ztop
    dz = np.zeros(altitude.shape)
    dz[0] = altitude[0]  # first level goes from 0 - z[0] so dz[0]=z[0]
    dz[1:] = np.diff(altitude)
    dz = np.expand_dims(dz, axis=(0,2))

    # integrate LWC for LWP
    lwp = (dz*lwc_cloud).sum(dim='TH_1_60_eta_theta', skipna=True, min_count=1)
    lwp = lwp.mean(dim='grid_number', skipna=True)
    #lwp = ma.mean(lwp.values, axis=1)
    model_lwp.append(lwp)

    # rad fluxes
    sw_down_flux = xr.open_dataarray(f'data/model/{sim.code}_m01s01i235_CAP.nc')
    lw_down_flux = xr.open_dataarray(f'data/model/{sim.code}_m01s02i207_CAP.nc')
    model_sw.append(sw_down_flux)
    model_lw.append(lw_down_flux)


## Plot figure 3

In [ ]:
fig = plt.figure(figsize=(20*fprops.in_cm,5*fprops.in_cm), dpi=fprops.dpi)
lwp_subset = [cap_lwp_1hr.sel(time=da.T1HR.values, method='nearest')
              for da in model_lwc]
flux_subset = [cap_rad_1hr.sel(time=da.T1HR_rad_diag.values, method='nearest')
               for da in model_sw]
ax1, ax2 = fig.subplots(1, 2, gridspec_kw={'width_ratios':[5,4],
                                           'wspace':0.025})
# ax1: CONTROL (ocean)
ax1.set_title('(a)', loc='left')
ax1.set_title('Ocean')
s = 1
sim = simulations[s]
ax1.plot(lwp_subset[s].time.values, lwp_subset[s].values, 'k', label='CAPRICORN-2', lw=1.2)
ax1.plot(model_lwc[s].T1HR.values, model_lwp[s]*1e3, label=sim.label,       
        color=sim.colour, lw=sim.linewidth)
ax1.set_ylabel(r'LWP [g m$^{{-2}}$]', fontsize=fprops.ax_label_fs)
# ax2: CONTROL (coast)
ax2.set_title('(b)', loc='left')
ax2.set_title('Coast')
s = 0
sim = simulations[s]
ax2.plot(lwp_subset[s].time.values, lwp_subset[s].values, 'k', label='CAPRICORN-2', lw=1.2)
ax2.plot(model_lwc[s].T1HR.values, model_lwp[s]*1e3, label=sim.label,
        color=sim.colour, lw=sim.linewidth)
xtick_fmt = '%d-%b'
for ax in [ax1,ax2]:
    ax.set_ylim(bottom=0, top=575)
    ax.tick_params(axis='x', labelsize=fprops.ax_fs)
    ax.tick_params(axis='y', labelsize=fprops.ax_fs)
    ax.xaxis.set_major_formatter(mdates.DateFormatter(xtick_fmt))
ax2.yaxis.set_ticklabels([])
ax2.legend(fontsize=fprops.legend_fs)
plt.savefig('figures/figure3.png', bbox_inches='tight', facecolor='white')


## Test sensivitity to RWP

In [ ]:
i_fig_sim = [0,1]

# load air density in kg m-3, to convert mass mixing ratio to concentrations
coast_rho = cfn.load_air_density(simulations[0].code, grid=True)

# load mass mixing ratio of liquid rain
coast_rain_mass = xr.open_dataarray(f'data/model/{simulations[0].code}_m01s00i272_grid_CAP.nc')
coast_rwc = coast_rain_mass*coast_rho.air_density

# calculate model z coord and z widths
altitude = coast_rwc.TH_1_60_eta_theta.values*const.ztop
dz = np.zeros(altitude.shape)
dz[0] = altitude[0]  # first level goes from 0 - z[0] so dz[0]=z[0]
dz[1:] = np.diff(altitude)
dz = np.expand_dims(dz, axis=(0,2))

# integrate for rain water path
coast_rwp = (dz*coast_rwc).sum(dim='TH_1_60_eta_theta', skipna=True, min_count=1)
coast_rwp = coast_rwp.mean(dim='grid_number', skipna=True)

# large scale surface rain rate
model_rain_rate = [xr.open_dataarray(f'data/model/{sim.code}_m01s04i203_CAP.nc') for sim in simulations[i_fig_sim]]


In [ ]:
fig = plt.figure(figsize=(20*fprops.in_cm,6*fprops.in_cm), dpi=fprops.dpi)
lwp_subset = [cap_lwp_1hr.sel(time=da.T1HR.values, method='nearest')
              for da in model_lwc]
rain_rate_subset = [cap_rain_rate_1hr.sel(time=da.T1HR.values, method='nearest')
                    for da in model_lwc]
flux_subset = [cap_rad_1hr.sel(time=da.T1HR_rad_diag.values, method='nearest')
               for da in model_sw]
ax2 = plt.axes()
ax2.set_title('Coast')
s = 0
sim = simulations[s]
ax2.plot(lwp_subset[s].time.values, lwp_subset[s].values, 'k', label='CAPRICORN-2', lw=1.2)
ax2.plot(model_lwc[s].T1HR.values, model_lwp[s]*1e3, label=sim.label,       
        color=sim.colour, lw=sim.linewidth)
ax2.plot(model_lwc[s].T1HR.values, (model_lwp[s]+coast_rwp)*1e3, label=f'{sim.label} LWP+RWP',
        color=sim.colour, lw=sim.linewidth, ls='dashed')
xtick_fmt = '%d-%b'
ax2.set_ylabel(r'LWP [g m$^{{-2}}$]', fontsize=fprops.ax_label_fs)
ax2.set_ylim(bottom=0)
ax2.tick_params(axis='x', labelsize=fprops.ax_fs)
ax2.tick_params(axis='y', labelsize=fprops.ax_fs)
ax2.xaxis.set_major_formatter(mdates.DateFormatter(xtick_fmt))
ax2.legend(fontsize=fprops.legend_fs)
plt.savefig('figures/lwp_sensitivity_to_rain.png', bbox_inches='tight', facecolor='white')


## Plot figure S4

In [ ]:
fig = plt.figure(figsize=(21*fprops.in_cm,25*fprops.in_cm), dpi=fprops.dpi)
i_fig_sim = [0,2,3,4,5,6]
labels = ['(a)','(b)','(c)','(d)','(e)']
axes = fig.subplots(5, 1)
s = i_fig_sim[0]
lwp0 = model_lwp[s]*1e3
for n,ax in enumerate(axes):
    s = i_fig_sim[n+1]
    sim = simulations[s]
    lwp_subset = cap_lwp_1hr.sel(time=model_lwc[s].T1HR.values, method='nearest')
    ax.tick_params(axis='both', labelsize=fprops.ax_fs)
    ax.plot(lwp_subset.time.values, lwp_subset.values, 'k', lw=1.2, label='CAPRICORN-2')
    ax.plot(model_lwc[i_fig_sim[0]].T1HR, 1e3*model_lwp[i_fig_sim[0]].values,
            c=simulations[i_fig_sim[0]].colour, lw=simulations[i_fig_sim[0]].linewidth)
    ax.plot(model_lwc[s].T1HR, 1e3*model_lwp[s].values,
            c=sim.colour, lw=sim.linewidth, label=sim.label)
    ax.set_title(sim.label)
    ax.legend(fontsize=fprops.legend_fs)
    xtick_fmt = '%d-%b'
    ax.xaxis.set_major_formatter(mdates.DateFormatter(xtick_fmt))
fig.supylabel('LWP [g m-2]', x=0.05, fontsize=fprops.label_fs)
fig.subplots_adjust(hspace=0.5)
plt.savefig('figures/figureS4.png', bbox_inches='tight', facecolor='white')


## Plot figure 4

In [ ]:
s = 0
sim = simulations[s]
fig = plt.figure(figsize=(20*fprops.in_cm,18*fprops.in_cm), dpi=fprops.dpi)
ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)
fig.subplots_adjust(hspace=0.5)
# Take subset of observations corresponding to this simulation:
cap_rad_subset = cap_rad_1hr.sel(time=model_sw[s].T1HR_rad_diag.values, method='nearest')
# Figure:
# ax1: SW down
ax1.plot(cap_rad_subset.time, cap_rad_subset.sw_down, 'k',
         label='CAPRICORN-2', zorder=1)
ax1.plot(model_sw[s].T1HR_rad_diag, model_sw[s], c=sim.colour, label=sim.label,
         lw=sim.linewidth, zorder=4)
ax1.set_ylabel('SW flux [W m$^{-2}$]', fontsize=fprops.ax_label_fs)
ax1.set_title('Downwelling SW', fontsize=fprops.label_fs)
ax1.tick_params(axis='x', labelsize=fprops.ax_fs)
ax1.tick_params(axis='y', labelsize=fprops.ax_fs)
ylim = ax1.get_ylim()
ax1.set_ylim(ylim[0], 950) # make room for legend
# ax2: LW down
ax2.plot(cap_rad_subset.time, cap_rad_subset.lw_down, 'k',
         label='CAPRICORN-2', zorder=1)
ax2.plot(model_lw[s].T1HR_rad_diag, model_lw[s], c=sim.colour, label=sim.label,
         lw=sim.linewidth, zorder=4)
ax2.set_title('Downwelling LW', fontsize=fprops.label_fs)
ax2.set_ylabel('LW flux [W m$^{-2}$]', fontsize=fprops.ax_label_fs)
ax2.tick_params(axis='x', labelsize=fprops.ax_fs)
ax2.tick_params(axis='y', labelsize=fprops.ax_fs)
# ax3: net incoming
ax3.plot(cap_rad_subset.time, (cap_rad_subset.lw_down+cap_rad_subset.sw_down), 'k',
         label='CAPRICORN-2', zorder=1)
ax3.plot(model_lw[s].T1HR_rad_diag, (model_sw[s]+model_lw[s]), c=sim.colour, label=sim.label,
         lw=sim.linewidth, zorder=4)
ax3.set_title('Total downwelling', fontsize=fprops.label_fs)
ax3.set_ylabel('Total flux [W m$^{-2}$]', fontsize=fprops.ax_label_fs)
ax3.tick_params(axis='x', labelsize=fprops.ax_fs)
ax3.tick_params(axis='y', labelsize=fprops.ax_fs)
ax1.legend(fontsize=fprops.legend_fs)
ax1.set_title('(a)', loc='left')
ax2.set_title('(b)', loc='left')
ax3.set_title('(c)', loc='left')
for ax in [ax1,ax2,ax3]:
    ax.xaxis.set_major_formatter(mdates.DateFormatter(xtick_fmt))
plt.savefig('figures/figure4.png', bbox_inches='tight',
            facecolor='white')


## Plot figure S5

In [ ]:
fig = plt.figure(figsize=(21*fprops.in_cm,25*fprops.in_cm), dpi=fprops.dpi)
i_fig_sim = [0,2,3,4,5,6]
labels = ['(a)','(b)','(c)','(d)','(e)']
axes = fig.subplots(5, 1)
s = i_fig_sim[0]
sw0 = model_sw[s]
for n,ax in enumerate(axes):
    s = i_fig_sim[n+1]
    sim = simulations[s]
    sw_subset = cap_rad_1hr.sel(time=model_sw[s].T1HR_rad_diag.values, method='nearest')
    ax.tick_params(axis='both', labelsize=fprops.ax_fs)
    ax.plot(sw_subset.time.values, sw_subset.sw_down.values, 'k', lw=1.2, label='CAPRICORN-2')
    ax.plot(model_sw[i_fig_sim[0]].T1HR_rad_diag, model_sw[i_fig_sim[0]].values,
            c=simulations[i_fig_sim[0]].colour, lw=simulations[i_fig_sim[0]].linewidth)
    ax.plot(model_sw[s].T1HR_rad_diag, model_sw[s].values,
            c=sim.colour, lw=sim.linewidth, label=sim.label)
    ax.set_title(sim.label)
    ax.legend(fontsize=fprops.legend_fs)
    ax.xaxis.set_major_formatter(mdates.DateFormatter(xtick_fmt))
fig.supylabel('Downwelling SW [W m-2]', x=0.05, fontsize=fprops.label_fs)
fig.subplots_adjust(hspace=0.5)
plt.savefig('figures/figureS5.png', bbox_inches='tight', facecolor='white')


## Plot figure S6

In [ ]:
fig = plt.figure(figsize=(21*fprops.in_cm,25*fprops.in_cm), dpi=fprops.dpi)
i_fig_sim = [0,2,3,4,5,6]
labels = ['(a)','(b)','(c)','(d)','(e)']
axes = fig.subplots(5, 1)
s = i_fig_sim[0]
lw0 = model_lw[s]
for n,ax in enumerate(axes):
    s = i_fig_sim[n+1]
    sim = simulations[s]
    lw_subset = cap_rad_1hr.sel(time=model_lw[s].T1HR_rad_diag.values, method='nearest')
    ax.tick_params(axis='both', labelsize=fprops.ax_fs)
    ax.plot(lw_subset.time.values, lw_subset.lw_down.values, 'k', lw=1.2, label='CAPRICORN-2')
    ax.plot(model_lw[i_fig_sim[0]].T1HR_rad_diag, model_lw[i_fig_sim[0]].values,
            c=simulations[i_fig_sim[0]].colour, lw=simulations[i_fig_sim[0]].linewidth)
    ax.plot(model_lw[s].T1HR_rad_diag, model_lw[s].values,
            c=sim.colour, lw=sim.linewidth, label=sim.label)
    ax.set_title(sim.label)
    ax.legend(fontsize=fprops.legend_fs)
    ax.xaxis.set_major_formatter(mdates.DateFormatter(xtick_fmt))
fig.supylabel('Downwelling LW [W m-2]', x=0.05, fontsize=fprops.label_fs)
fig.subplots_adjust(hspace=0.5)
plt.savefig('figures/figureS6.png', bbox_inches='tight', facecolor='white')


## Plot figure 5

In [ ]:
ifig_sims = [0,2,3,4,5,6]
# Calculate bins for distributions of LWP and radiation fluxes
lwp_subset = cap_lwp_1hr.sel(time=model_lwp[ifig_sims[0]].T1HR.values, method='nearest')
lwp_subset = lwp_subset.where(lwp_subset>0)
rad_subset = cap_rad_1hr.sel(time=model_sw[ifig_sims[0]].T1HR_rad_diag.values, method='nearest')
cap_lwp_bins, cap_lwp_stats = get_hist_bins(lwp_subset)
cap_sw_bins, cap_sw_stats = get_hist_bins(rad_subset.sw_down)
cap_lw_bins, cap_lw_stats = get_hist_bins(rad_subset.lw_down)
cap_lwp_cts, cap_lwp_bins = np.histogram(ma.masked_invalid(lwp_subset).compressed(),
                                         bins=cap_lwp_bins, density=True)
cap_sw_cts, cap_sw_bins = np.histogram(ma.masked_invalid(rad_subset.sw_down).compressed(),
                                         bins=cap_sw_bins, density=True)
cap_lw_cts, cap_lw_bins = np.histogram(ma.masked_invalid(rad_subset.lw_down).compressed(),
                                         bins=cap_lw_bins, density=True)

for s in ifig_sims:
    sim = simulations[s]
    lwp_bins, lwp_stats = get_hist_bins(model_lwp[s]*1000)
    sw_bins, sw_stats = get_hist_bins(model_sw[s])
    lw_bins, lw_stats = get_hist_bins(model_lw[s])
    lwp_cts, lwp_bins = np.histogram(ma.masked_invalid(model_lwp[s]*1000).compressed(),
                                             bins=lwp_bins, density=True)
    sw_cts, sw_bins = np.histogram(ma.masked_invalid(model_sw[s]).compressed(),
                                             bins=sw_bins, density=True)
    lw_cts, lw_bins = np.histogram(ma.masked_invalid(model_lw[s]).compressed(),
                                             bins=lw_bins, density=True)


# plot
fig = plt.figure(figsize=(21*fprops.in_cm,18*fprops.in_cm), dpi=300)
axes = fig.subplots(3, 3)
iax_sims = [[0,6],[2,3],[4,5]]
for n,ax in enumerate(axes[:,0]):
    ax.stairs(cap_lwp_cts, cap_lwp_bins, color='k', lw=1.2, label='CAPRICORN-2')
    for s in iax_sims[n]:
        sim = simulations[s]
        lwp_bins, lwp_stats = get_hist_bins(model_lwp[s]*1000)
        lwp_cts, lwp_bins = np.histogram(ma.masked_invalid(model_lwp[s]*1000).compressed(),
                                         bins=lwp_bins, density=True)
        ax.stairs(lwp_cts, lwp_bins, color=sim.colour, lw=sim.linewidth, label=sim.label)
        ax.legend(fontsize=fprops.legend_fs)
for n,ax in enumerate(axes[:,1]):
    ax.stairs(cap_sw_cts, cap_sw_bins, color='k', lw=1.2)
    for s in iax_sims[n]:
        sim = simulations[s]
        sw_bins, sw_stats = get_hist_bins(model_sw[s])
        sw_cts, sw_bins = np.histogram(ma.masked_invalid(model_sw[s]).compressed(),
                                         bins=sw_bins, density=True)
        ax.stairs(sw_cts, sw_bins, color=sim.colour, lw=sim.linewidth, label=sim.label)
for n,ax in enumerate(axes[:,2]):
    ax.stairs(cap_lw_cts, cap_lw_bins, color='k', lw=1.2)
    for s in iax_sims[n]:
        sim = simulations[s]
        lw_bins, lw_stats = get_hist_bins(model_lw[s])
        lw_cts, lw_bins = np.histogram(ma.masked_invalid(model_lw[s]).compressed(),
                                         bins=lw_bins, density=True)
        ax.stairs(lw_cts, lw_bins, color=sim.colour, lw=sim.linewidth, label=sim.label)
lwp_labels = ['(a)','(d)','(g)']
sw_labels =  ['(b)','(e)','(h)']
lw_labels =  ['(c)','(f)','(i)']
for n,ax in enumerate(axes[:,0]):
    ax.set_ylim(0,0.04)
    ax.tick_params(axis='both', labelsize=fprops.ax_fs)
    yticks = ax.get_yticks()
    ax.set_title(lwp_labels[n], fontsize=fprops.label_fs, loc='left')
axes[2,0].set_xlabel('LWP [g m$^{{-2}}$]', fontsize=fprops.ax_label_fs)
for n,ax in enumerate(axes[:,1]):
    ax.set_ylim(0,0.0075)
    ax.tick_params(axis='both', labelsize=fprops.ax_fs)
    yticks = ax.get_yticks()
    ax.set_title(sw_labels[n], fontsize=fprops.label_fs, loc='left')
axes[2,1].set_xlabel('Flux [W m$^{{-2}}$]', fontsize=fprops.ax_label_fs)
for n,ax in enumerate(axes[:,2]):
    ax.set_ylim(0,0.035)
    ax.tick_params(axis='both', labelsize=fprops.ax_fs)
    yticks = ax.get_yticks()
    ax.set_title(lw_labels[n], fontsize=fprops.label_fs, loc='left')
axes[2,2].set_xlabel('Flux [W m$^{{-2}}$]', fontsize=fprops.ax_label_fs)
fig.supylabel('PDF', fontsize=fprops.ax_label_fs, x=0.05)
fig.subplots_adjust(wspace=0.4, hspace=0.35)
plt.savefig('figures/figure5.png', bbox_inches='tight', facecolor='white')


## Output statistics of distributions

In [ ]:
print('LWP STATISTICS')
print(f'{"":10s} {"MIN":>6s}, {"MAX":>6s}, {"MEAN":>6s}, MEDIAN')
print(f'{"CAP":>9s}: {cap_lwp_stats[0]: >6.2f}, {cap_lwp_stats[1]:>6.2f}, {cap_lwp_stats[2]: >6.2f}, {cap_lwp_stats[3]:>6.2f}')
for s in [0,2,3,4,5,6]:
    sim = simulations[s]
    lwp_stats = get_hist_bins(1000*model_lwp[s])[1]
    print(f'{sim.label:>9.9s}: {lwp_stats[0]: >6.2f}, {lwp_stats[1]:>6.2f}, {lwp_stats[2]: >6.2f}, {lwp_stats[3]:>6.2f}')

print('SW STATISTICS')
print(f'{"":10s} {"MIN":>6s}, {"MAX":>6s}, {"MEAN":>6s}, MEDIAN')
print(f'{"CAP":>9s}: {cap_sw_stats[0]: >6.2f}, {cap_sw_stats[1]:>6.2f}, {cap_sw_stats[2]: >6.2f}, {cap_sw_stats[3]:>6.2f}')
for s in [0,2,3,4,5,6]:
    sim = simulations[s]
    sw_stats = get_hist_bins(model_sw[s])[1]
    print(f'{sim.label:>9.9s}: {sw_stats[0]: >6.2f}, {sw_stats[1]:>6.2f}, {sw_stats[2]: >6.2f}, {sw_stats[3]:>6.2f}')

print('LW STATISTICS')
print(f'{"":10s} {"MIN":>6s}, {"MAX":>6s}, {"MEAN":>6s}, MEDIAN')
print(f'{"CAP":>9s}: {cap_lw_stats[0]: >6.2f}, {cap_lw_stats[1]:>6.2f}, {cap_lw_stats[2]: >6.2f}, {cap_lw_stats[3]:>6.2f}')
for s in [0,2,3,4,5,6]:
    sim = simulations[s]
    lw_stats = get_hist_bins(model_lw[s])[1]
    print(f'{sim.label:>9.9s}: {lw_stats[0]: >6.2f}, {lw_stats[1]:>6.2f}, {lw_stats[2]: >6.2f}, {lw_stats[3]:>6.2f}')
